# Recursive LLM Chess Tree Search Demo

This notebook demonstrates the recursive LLM tree search framework for chess analysis. It allows you to explore a chess position using multiple LLM instances that can recursively call each other to explore different branches of the game tree.

In [ ]:
# Setup and imports
import sys
import os
import chess
import pandas as pd
import matplotlib.pyplot as plt

# Add project directory to path
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Import tree_games modules
from tree_games.chess_engine.core import ChessEngine
from tree_games.tree_search.controller import TreeSearchController
from tree_games.utils.visualization import display_board, display_evaluation, display_comparison_table

# Create mock LLM API caller for demonstration
from tree_games.chess_cli import MockLLMAPI
mock_llm_api = MockLLMAPI()

## Initialize the Chess Engine

First, let's set up the chess engine and display the initial board position.

In [ ]:
# Create chess engine
engine = ChessEngine()

# Display the initial board position
display_board(engine.board)

## Initialize the Tree Search Controller

Now let's create the tree search controller that will manage the exploration of different variations.

In [ ]:
# Define a simple LLM caller function that uses the mock API
def llm_caller(branch_context):
    # Extract board and other context
    board = branch_context.get("board")
    move_path = branch_context.get("move_path", [])
    
    # Create a simple prompt
    prompt = f"Board State:\n{board}\n\nMoves: {move_path}\n\nPlease analyze this position."
    
    # Call mock API
    response = mock_llm_api(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert chess analyst."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Extract and parse response
    content = response.get("choices", [{}])[0].get("message", {}).get("content", "{}")
    
    # Try to parse as JSON
    import json
    try:
        evaluation = json.loads(content)
        return evaluation
    except json.JSONDecodeError:
        # Fallback evaluation
        return {
            "position_score": 0.0,
            "confidence": 0.5,
            "material_balance": 0.0,
            "development_score": 0.0,
            "tactical_score": 0.0,
            "plans": ["JSON parsing failed"],
            "key_variations": ["Error in LLM response"]
        }

# Create tree search controller with LLM caller
controller = TreeSearchController(llm_caller=llm_caller)

# Set exploration parameters
controller.max_branches = 3  # Analyze top 3 moves at each position
controller.max_depth = 3     # Explore up to 3 moves deep

## Analyze the Initial Position

Let's explore the top candidate moves from the initial position.

In [ ]:
# Evaluate all top moves
results = controller.evaluate_all_moves(max_branches=3)

# Display results as a comparison table
display_comparison_table(results.get("evaluations", []))

## Explore a Specific Branch

Now let's explore a specific opening move in more depth.

In [ ]:
# Choose a move to explore (e4)
move = chess.Move.from_uci("e2e4")

# Explore the branch
evaluation = controller.explore_branch(move, max_branches=3)

# Display the evaluation
display_evaluation(evaluation)

## Make a Move and Continue Analysis

Let's execute the move we just explored and continue analyzing from the new position.

In [ ]:
# Execute the move
controller.commit_move(move)

# Display the new board position
display_board(controller.engine.board)

# Analyze the new position
results = controller.evaluate_all_moves(max_branches=3)

# Display results
display_comparison_table(results.get("evaluations", []))

## Explore a Sub-Branch

Let's explore a specific response to the move we just played.

In [ ]:
# Choose a response move (e5 - the Ruy Lopez)
response_move = chess.Move.from_uci("e7e5")

# Explore the sub-branch
sub_evaluation = controller.explore_branch(response_move, max_branches=2)

# Display the evaluation
display_evaluation(sub_evaluation)

## Simulation: Play Out a Complete Game

Let's simulate a complete game by selecting the best move at each position.

In [ ]:
# Reset the controller
controller.reset()

# Display the starting position
print("Starting position:")
display_board(controller.engine.board)

# Play 10 moves or until game over
for move_num in range(1, 11):
    if controller.engine.is_game_over():
        print(f"Game over: {controller.engine.get_result()}")
        break
        
    # Analyze position
    controller.evaluate_all_moves(max_branches=2)
    
    # Get best move
    best_move = controller.get_best_move()
    
    if best_move is None:
        print("No moves available")
        break
    
    # Execute best move
    san = controller.engine.board.san(best_move)
    turn = "White" if controller.engine.board.turn == chess.WHITE else "Black"
    controller.commit_move(best_move)
    
    print(f"Move {move_num}: {turn} plays {san}")
    
# Display final position
print("\nFinal position:")
display_board(controller.engine.board)

## Exploring Advanced Features

The tree search framework supports more advanced features like visualization of the search tree, exporting analysis to PGN, and more. These features are available through the utility modules, but they require additional dependencies like NetworkX and Graphviz.

## Conclusion

This notebook has demonstrated the basic functionality of the recursive LLM chess tree search system. The key advantage of this approach is that it can distribute reasoning across multiple LLM instances, each specializing in exploring a specific branch of the game tree. This allows for more comprehensive exploration of chess positions, resulting in better move decisions.

For real-world usage, you would replace the mock LLM API with actual calls to an LLM API like OpenAI's GPT-4 or Anthropic's Claude.